In [1]:
import pickle
import os
import numpy as np
import pandas as pd
import music21 as m21
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
def chooseOneGPU(chosenGPU):
    '''
    Limits tensorflow to one GPU
    Inputs:
        chosenGPU: the ID of the chosen GPU
    Outputs:
        none
    '''
    # Limit to ONE GPU
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        # Restrict TensorFlow to only use ONE GPU
        try:
            tf.config.experimental.set_visible_devices(gpus[chosenGPU], 'GPU')
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
        except RuntimeError as e:
            # Visible devices must be set before GPUs have been initialized
            print(e)

In [4]:
print(tf.config.experimental.list_physical_devices('GPU'))
chooseOneGPU(int(input("Choose the ID of the GPU you want to use: ")))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Choose the ID of the GPU you want to use: 0
1 Physical GPUs, 1 Logical GPU


In [5]:
def openScore (fileName, folder = 'databases/nesmdb24_seprsco/train/'):
    with open(folder+fileName, 'rb') as f:
        rate, nsamps, seprsco = pickle.load(f)
    return rate, nsamps, seprsco

In [6]:
def openFolder (folder):
    i = 0
    df = pd.DataFrame(columns = ("score", "rate", "nsamps", "fileName", "notes"))
    for file in os.listdir(folder):
        rate, nsamps, seprsco = openScore(file, folder)
        notes, dump = seprsco.shape
        df.loc[i] = [seprsco, rate, nsamps, file, notes]
        i+=1
    df['rate'] = pd.to_numeric(df['rate'])
    df['nsamps'] = pd.to_numeric(df['nsamps'])
    df['notes'] = pd.to_numeric(df['notes'])
    return df

In [7]:
def openFolderSplit (folder, splitOn = 100):
    i = 0
    df = pd.DataFrame(columns = ("score", "rate", "origNsamps", "fileName"))
    for file in os.listdir(folder):
        rate, nsamps, seprsco = openScore(file, folder)
        notes, dump = seprsco.shape
        j = 0;
        while (notes > (j+1)*splitOn):
            #should split on number of notes 100
            print ("added:", file, "notes:", str(j*splitOn), "-", str((j+1)*splitOn))
            df.loc[i] = [seprsco[j*splitOn:(j+1)*splitOn], rate, nsamps, file+str(j)]
            i+=1
            j+=1
    df['rate'] = pd.to_numeric(df['rate'])
    df['origNsamps'] = pd.to_numeric(df['origNsamps'])
    return df

In [8]:
def openFolderSplit2 (folder, splitOn = 100, normalize=True):
    channelLengths = [108, 108, 108, 16]
    items = []
    for file in os.listdir(folder):
        rate, nsamps, seprsco = openScore(file, folder)
        if (normalize):
            seprsco = seprsco.astype(float)
            for i in range(4):
                seprsco[:,i] =(seprsco[:, i] - channelLengths[i]/2)/(channelLengths[i]/2)
        j = 0;
        while (len(seprsco) > (j+1)*splitOn):
            #should split on number of notes 100
            items.append(seprsco[j*splitOn:(j+1)*splitOn])
            print ("added:", file, "notes:", str(j*splitOn), "-", str((j+1)*splitOn-1))
            j+=1
    return np.array(items)

In [9]:
items = openFolderSplit2('databases/nesmdb24_seprsco/train/', 1024)
items.shape

added: 005_Abadox_TheDeadlyInnerWar_01_02OpeningDemo.seprsco.pkl notes: 0 - 1023
added: 005_Abadox_TheDeadlyInnerWar_02_03Stage146.seprsco.pkl notes: 0 - 1023
added: 005_Abadox_TheDeadlyInnerWar_04_05Stage3.seprsco.pkl notes: 0 - 1023
added: 005_Abadox_TheDeadlyInnerWar_08_09Rescued.seprsco.pkl notes: 0 - 1023
added: 005_Abadox_TheDeadlyInnerWar_09_10Escape.seprsco.pkl notes: 0 - 1023
added: 005_Abadox_TheDeadlyInnerWar_10_11Ending.seprsco.pkl notes: 0 - 1023
added: 005_Abadox_TheDeadlyInnerWar_10_11Ending.seprsco.pkl notes: 1024 - 2047
added: 007_AfterBurner_01_02FinalTakeOff.seprsco.pkl notes: 0 - 1023
added: 007_AfterBurner_01_02FinalTakeOff.seprsco.pkl notes: 1024 - 2047
added: 007_AfterBurner_01_02FinalTakeOff.seprsco.pkl notes: 2048 - 3071
added: 007_AfterBurner_01_02FinalTakeOff.seprsco.pkl notes: 3072 - 4095
added: 007_AfterBurner_02_03RedOut.seprsco.pkl notes: 0 - 1023
added: 007_AfterBurner_02_03RedOut.seprsco.pkl notes: 1024 - 2047
added: 007_AfterBurner_03_04SuperStripe.sep

added: 043_Castelian_03_04BonusRoad.seprsco.pkl notes: 0 - 1023
added: 043_Castelian_03_04BonusRoad.seprsco.pkl notes: 1024 - 2047
added: 043_Castelian_03_04BonusRoad.seprsco.pkl notes: 2048 - 3071
added: 044_CastleofDragon_02_03Map.seprsco.pkl notes: 0 - 1023
added: 044_CastleofDragon_02_03Map.seprsco.pkl notes: 1024 - 2047
added: 044_CastleofDragon_02_03Map.seprsco.pkl notes: 2048 - 3071
added: 044_CastleofDragon_13_14CreditRoll.seprsco.pkl notes: 0 - 1023
added: 044_CastleofDragon_13_14CreditRoll.seprsco.pkl notes: 1024 - 2047
added: 045_Castlevania_02_03Stalker.seprsco.pkl notes: 0 - 1023
added: 045_Castlevania_03_04WickedChild.seprsco.pkl notes: 0 - 1023
added: 045_Castlevania_05_06HeartofFire.seprsco.pkl notes: 0 - 1023
added: 045_Castlevania_05_06HeartofFire.seprsco.pkl notes: 1024 - 2047
added: 045_Castlevania_15_16Voyager.seprsco.pkl notes: 0 - 1023
added: 046_CastlevaniaIII_Dracula_sCurse_00_01Prelude.seprsco.pkl notes: 0 - 1023
added: 046_CastlevaniaIII_Dracula_sCurse_00_01P

added: 066_DestinyofanEmperor_11_12Field2.seprsco.pkl notes: 0 - 1023
added: 066_DestinyofanEmperor_15_16Dungeon.seprsco.pkl notes: 0 - 1023
added: 066_DestinyofanEmperor_15_16Dungeon.seprsco.pkl notes: 1024 - 2047
added: 066_DestinyofanEmperor_17_18Luoyang.seprsco.pkl notes: 0 - 1023
added: 066_DestinyofanEmperor_18_19Ending.seprsco.pkl notes: 0 - 1023
added: 072_DonDokoDon_02_03SylvanLand.seprsco.pkl notes: 0 - 1023
added: 072_DonDokoDon_06_07Dungeon.seprsco.pkl notes: 0 - 1023
added: 072_DonDokoDon_08_09BadEnd.seprsco.pkl notes: 0 - 1023
added: 072_DonDokoDon_08_09BadEnd.seprsco.pkl notes: 1024 - 2047
added: 072_DonDokoDon_08_09BadEnd.seprsco.pkl notes: 2048 - 3071
added: 072_DonDokoDon_09_10CastleinTheSky.seprsco.pkl notes: 0 - 1023
added: 072_DonDokoDon_10_11HappyEnd.seprsco.pkl notes: 0 - 1023
added: 072_DonDokoDon_10_11HappyEnd.seprsco.pkl notes: 1024 - 2047
added: 072_DonDokoDon_10_11HappyEnd.seprsco.pkl notes: 2048 - 3071
added: 076_DonkeyKongCountry4_06_07AquaticAmbiance.sepr

added: 118_FinalFantasyII_13_14Dungeon.seprsco.pkl notes: 0 - 1023
added: 118_FinalFantasyII_15_16BattleTheme2.seprsco.pkl notes: 0 - 1023
added: 118_FinalFantasyII_17_18Finale.seprsco.pkl notes: 0 - 1023
added: 118_FinalFantasyII_17_18Finale.seprsco.pkl notes: 1024 - 2047
added: 118_FinalFantasyII_17_18Finale.seprsco.pkl notes: 2048 - 3071
added: 118_FinalFantasyII_17_18Finale.seprsco.pkl notes: 3072 - 4095
added: 119_FinalFantasyIII_01_02CrystalCave.seprsco.pkl notes: 0 - 1023
added: 119_FinalFantasyIII_02_03Battle1.seprsco.pkl notes: 0 - 1023
added: 119_FinalFantasyIII_05_06OpeningTheme.seprsco.pkl notes: 0 - 1023
added: 119_FinalFantasyIII_06_07MyHomeTown.seprsco.pkl notes: 0 - 1023
added: 119_FinalFantasyIII_07_08EternalWind.seprsco.pkl notes: 0 - 1023
added: 119_FinalFantasyIII_16_17TimeRemains.seprsco.pkl notes: 0 - 1023
added: 119_FinalFantasyIII_21_22CastleofHain.seprsco.pkl notes: 0 - 1023
added: 119_FinalFantasyIII_25_26Battle2.seprsco.pkl notes: 0 - 1023
added: 119_FinalFan

added: 135_GanbareGoemonGaiden2_TenkanoZaih__39_40WesternHouseForeignShip.seprsco.pkl notes: 0 - 1023
added: 135_GanbareGoemonGaiden2_TenkanoZaih__39_40WesternHouseForeignShip.seprsco.pkl notes: 1024 - 2047
added: 135_GanbareGoemonGaiden2_TenkanoZaih__41_42TownHarbor.seprsco.pkl notes: 0 - 1023
added: 135_GanbareGoemonGaiden2_TenkanoZaih__94_95EpiloguePart2.seprsco.pkl notes: 0 - 1023
added: 135_GanbareGoemonGaiden2_TenkanoZaih__95_96StaffRoll.seprsco.pkl notes: 0 - 1023
added: 137_GansoSaiyuuki_SuperMonkeyDaibouken_00_01TitleScreen.seprsco.pkl notes: 0 - 1023
added: 137_GansoSaiyuuki_SuperMonkeyDaibouken_01_02Overworld.seprsco.pkl notes: 0 - 1023
added: 137_GansoSaiyuuki_SuperMonkeyDaibouken_01_02Overworld.seprsco.pkl notes: 1024 - 2047
added: 137_GansoSaiyuuki_SuperMonkeyDaibouken_02_03Fighting.seprsco.pkl notes: 0 - 1023
added: 137_GansoSaiyuuki_SuperMonkeyDaibouken_03_04Houses.seprsco.pkl notes: 0 - 1023
added: 137_GansoSaiyuuki_SuperMonkeyDaibouken_03_04Houses.seprsco.pkl notes: 1

...

added: 298_SolarJetman_HuntfortheGoldenWarpship_00_01MainTheme.seprsco.pkl notes: 0 - 1023
added: 298_SolarJetman_HuntfortheGoldenWarpship_03_04PreludonGameplay.seprsco.pkl notes: 0 - 1023
added: 298_SolarJetman_HuntfortheGoldenWarpship_03_04PreludonGameplay.seprsco.pkl notes: 1024 - 2047
added: 298_SolarJetman_HuntfortheGoldenWarpship_03_04PreludonGameplay.seprsco.pkl notes: 2048 - 3071
added: 298_SolarJetman_HuntfortheGoldenWarpship_03_04PreludonGameplay.seprsco.pkl notes: 3072 - 4095
added: 298_SolarJetman_HuntfortheGoldenWarpship_03_04PreludonGameplay.seprsco.pkl notes: 4096 - 5119
added: 298_SolarJetman_HuntfortheGoldenWarpship_03_04PreludonGameplay.seprsco.pkl notes: 5120 - 6143
added: 298_SolarJetman_HuntfortheGoldenWarpship_03_04PreludonGameplay.seprsco.pkl notes: 6144 - 7167
added: 298_SolarJetman_HuntfortheGoldenWarpship_03_04PreludonGameplay.seprsco.pkl notes: 7168 - 8191
added: 298_SolarJetman_HuntfortheGoldenWarpship_03_04PreludonGameplay.seprsco.pkl notes: 8192 - 9215
add

added: 320_SuperHIK300in1_00_01Menu.seprsco.pkl notes: 0 - 1023
added: 321_SuperLodeRunner_04_05Unknown2.seprsco.pkl notes: 0 - 1023
added: 321_SuperLodeRunner_06_07Unknown4.seprsco.pkl notes: 0 - 1023
added: 322_SuperMarioBros__00_01RunningAbout.seprsco.pkl notes: 0 - 1023
added: 322_SuperMarioBros__00_01RunningAbout.seprsco.pkl notes: 1024 - 2047
added: 322_SuperMarioBros__11_12RunningAboutHurry.seprsco.pkl notes: 0 - 1023
added: 325_SuperMarioWorld_04_05Fortress.seprsco.pkl notes: 0 - 1023
added: 325_SuperMarioWorld_04_05Fortress.seprsco.pkl notes: 1024 - 2047
added: 325_SuperMarioWorld_06_07Underworld.seprsco.pkl notes: 0 - 1023
added: 325_SuperMarioWorld_07_08Swimming.seprsco.pkl notes: 0 - 1023
added: 325_SuperMarioWorld_08_09HauntedHouse.seprsco.pkl notes: 0 - 1023
added: 325_SuperMarioWorld_12_13GameClear.seprsco.pkl notes: 0 - 1023
added: 327_SuperStarForce_JikuurekinoHimitsu_00_01TakeOffDA20101608.seprsco.pkl notes: 0 - 1023
added: 327_SuperStarForce_JikuurekinoHimitsu_00_01T

added: 375_Vs_Excitebike_20_21TrackMode.seprsco.pkl notes: 0 - 1023
added: 378_WaiWaiWorld2_SOS__ParsleyJou_00_01Opening.seprsco.pkl notes: 0 - 1023
added: 378_WaiWaiWorld2_SOS__ParsleyJou_04_05RunRickle1.seprsco.pkl notes: 0 - 1023
added: 378_WaiWaiWorld2_SOS__ParsleyJou_05_06RickleShooterGO1.seprsco.pkl notes: 0 - 1023
added: 378_WaiWaiWorld2_SOS__ParsleyJou_06_07RickleShooterGO2.seprsco.pkl notes: 0 - 1023
added: 378_WaiWaiWorld2_SOS__ParsleyJou_10_11BlossomsofOedo2.seprsco.pkl notes: 0 - 1023
added: 378_WaiWaiWorld2_SOS__ParsleyJou_13_14TwinBeesDeparture12.seprsco.pkl notes: 0 - 1023
added: 378_WaiWaiWorld2_SOS__ParsleyJou_13_14TwinBeesDeparture12.seprsco.pkl notes: 1024 - 2047
added: 378_WaiWaiWorld2_SOS__ParsleyJou_18_19FightingJungle1.seprsco.pkl notes: 0 - 1023
added: 378_WaiWaiWorld2_SOS__ParsleyJou_21_22SweetsDance2.seprsco.pkl notes: 0 - 1023
added: 378_WaiWaiWorld2_SOS__ParsleyJou_27_28InvitationtotheWorldofDemons1.seprsco.pkl notes: 0 - 1023
added: 378_WaiWaiWorld2_SOS__Pa

added: 398_Ys_AncientYsVanishedOmen_07_08Palace.seprsco.pkl notes: 0 - 1023
added: 398_Ys_AncientYsVanishedOmen_07_08Palace.seprsco.pkl notes: 1024 - 2047
added: 398_Ys_AncientYsVanishedOmen_08_09PalaceB3F.seprsco.pkl notes: 0 - 1023
added: 398_Ys_AncientYsVanishedOmen_10_11BeatofTheTerror.seprsco.pkl notes: 0 - 1023
added: 398_Ys_AncientYsVanishedOmen_11_12TowerofTheShadowofDeath1.seprsco.pkl notes: 0 - 1023
added: 398_Ys_AncientYsVanishedOmen_12_13TowerofTheShadowofDeath2.seprsco.pkl notes: 0 - 1023
added: 398_Ys_AncientYsVanishedOmen_13_14TowerofTheShadowofDeath3.seprsco.pkl notes: 0 - 1023
added: 398_Ys_AncientYsVanishedOmen_14_15TowerofTheShadowofDeath4.seprsco.pkl notes: 0 - 1023
added: 398_Ys_AncientYsVanishedOmen_15_16TowerofTheShadowofDeath5.seprsco.pkl notes: 0 - 1023
added: 398_Ys_AncientYsVanishedOmen_16_17TowerofTheShadowofDeath6.seprsco.pkl notes: 0 - 1023
added: 398_Ys_AncientYsVanishedOmen_17_18TheLastMomentofTheDark.seprsco.pkl notes: 0 - 1023
added: 398_Ys_AncientYsVa

(1700, 1024, 4)

In [10]:
np.savetxt("foo.csv", items[0], delimiter=",")

In [11]:
itemsTest = openFolderSplit2('databases/nesmdb24_seprsco/test/', 1024)
itemsTest.shape

added: 002_1943_TheBattleofMidway_03_04AirBattleA.seprsco.pkl notes: 0 - 1023
added: 002_1943_TheBattleofMidway_04_05AntishipBattleA.seprsco.pkl notes: 0 - 1023
added: 002_1943_TheBattleofMidway_08_09AntishipBattleC.seprsco.pkl notes: 0 - 1023
added: 002_1943_TheBattleofMidway_09_10AirBattleD.seprsco.pkl notes: 0 - 1023
added: 002_1943_TheBattleofMidway_13_14Ending.seprsco.pkl notes: 0 - 1023
added: 013_AlterEgo_01_02Level1.seprsco.pkl notes: 0 - 1023
added: 013_AlterEgo_02_03Level2.seprsco.pkl notes: 0 - 1023
added: 013_AlterEgo_03_04Level2Alternate.seprsco.pkl notes: 0 - 1023
added: 037_BoobyKids_02_03BoobySamba.seprsco.pkl notes: 0 - 1023
added: 037_BoobyKids_05_06FunnyDragon.seprsco.pkl notes: 0 - 1023
added: 037_BoobyKids_08_09WildStreet.seprsco.pkl notes: 0 - 1023
added: 037_BoobyKids_10_11BoobieWoogie.seprsco.pkl notes: 0 - 1023
added: 038_BubbleBobble_00_01IntroductionMainTheme.seprsco.pkl notes: 0 - 1023
added: 038_BubbleBobble_02_03SecretRoom.seprsco.pkl notes: 0 - 1023
added

(83, 1024, 4)

In [12]:
itemsVal = openFolderSplit2('databases/nesmdb24_seprsco/valid/', 1024)
itemsVal.shape

added: 029_Battleship_00_01TitleScreen.seprsco.pkl notes: 0 - 1023
added: 029_Battleship_01_02SetUpTheme.seprsco.pkl notes: 0 - 1023
added: 029_Battleship_06_07UnusedSong1.seprsco.pkl notes: 0 - 1023
added: 029_Battleship_08_09UnusedSong3.seprsco.pkl notes: 0 - 1023
added: 054_ChukaTaisen_09_10Ending.seprsco.pkl notes: 0 - 1023
added: 054_ChukaTaisen_09_10Ending.seprsco.pkl notes: 1024 - 2047
added: 079_DoubleDragonIII_TheSacredStones_00_01Title.seprsco.pkl notes: 0 - 1023
added: 079_DoubleDragonIII_TheSacredStones_08_09ChinsThemeBoss2.seprsco.pkl notes: 0 - 1023
added: 079_DoubleDragonIII_TheSacredStones_09_10InJapanMission31.seprsco.pkl notes: 0 - 1023
added: 079_DoubleDragonIII_TheSacredStones_12_13InItalyMission4.seprsco.pkl notes: 0 - 1023
added: 079_DoubleDragonIII_TheSacredStones_12_13InItalyMission4.seprsco.pkl notes: 1024 - 2047
added: 079_DoubleDragonIII_TheSacredStones_14_15EntertheDragonMission51.seprsco.pkl notes: 0 - 1023
added: 079_DoubleDragonIII_TheSacredStones_15_16Cu

added: 394_Yoshi_sCookie_03_041PMusicTypeB.seprsco.pkl notes: 0 - 1023
added: 399_YumeKoujou_DokiDokiPanic_12_13Ending.seprsco.pkl notes: 0 - 1023
added: 402_ZeldanoDensetsu_00_01TitleBGM.seprsco.pkl notes: 0 - 1023
added: 402_ZeldanoDensetsu_08_09EndingTheme.seprsco.pkl notes: 0 - 1023
added: 402_ZeldanoDensetsu_08_09EndingTheme.seprsco.pkl notes: 1024 - 2047


(122, 1024, 4)

In [13]:
itemsAll = np.concatenate([items, itemsTest, itemsVal])

In [14]:
itemsAll.shape

(1905, 1024, 4)

In [15]:
class NESGAN:
    
    mus_length = 1024
    
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1024
        self.disc_loss = []
        self.gen_loss = []
        
        optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)
        
        self.discriminator = self.createDiscriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
        self.generator = self.createGenerator()
        
        z = tf.keras.layers.Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)
        
        self.discriminator.trainable = False
        
        validity = self.discriminator(generated_seq)
        
        self.combined = tf.keras.models.Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    def createDiscriminator(self):
        model = tf.keras.models.Sequential([
            tf.keras.layers.LSTM(512, input_shape = self.seq_shape, return_sequences=True),
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512)),
            tf.keras.layers.Dense(512),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            tf.keras.layers.Dense(256),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
        model.summary()
        seq = tf.keras.layers.Input(self.seq_shape)
        validity = model(seq)
        
        return tf.keras.models.Model(seq, validity)
    
    def createGenerator(self):
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(256, input_dim = self.latent_dim),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            tf.keras.layers.BatchNormalization(momentum=0.8),
            tf.keras.layers.Dense(512),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            tf.keras.layers.BatchNormalization(momentum=0.8),
            tf.keras.layers.Dense(1024),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            tf.keras.layers.BatchNormalization(momentum=0.8),
            tf.keras.layers.Dense(np.prod(self.seq_shape), activation='tanh'),
            tf.keras.layers.Reshape(self.seq_shape)
        ])
        
        model.summary()
        
        noise = tf.keras.layers.Input(shape=(self.latent_dim,))
        seq=model(noise)
        
        return tf.keras.models.Model(noise, seq)
    
    def train(self, x_train, epochs, batch_size=128, sample_interval=50):
        
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        for epoch in range(epochs):
            
            idx = np.random.randint(0, x_train.shape[0], batch_size)
            real_seqs = x_train[idx]
            
            noise = tf.random.normal((batch_size, self.latent_dim), 0, 1)
            
            gan_seqs = self.generator.predict(noise)
            
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            noise = tf.random.normal((batch_size, self.latent_dim), 0, 1)
            
            g_loss = self.combined.train_on_batch(noise, real)
            
            if (epoch % sample_interval == 0):
                print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.disc_loss.append(d_loss[0])
                self.gen_loss.append(g_loss)
            
        #self.generate(notes)
        self.plot_loss()
    
    def generate(self, input_notes):
        
        noise = tf.random.normal((1, self.latent_dim), 0, 1)
        predictions = self.generator.predict(noise)
    
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_final.png', transparent=True)
        plt.close()
        
        
        
        

In [16]:
gan = NESGAN(rows=1024)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1024, 512)         1052672   
_________________________________________________________________
bidirectional (Bidirectional (None, 1024)              4198400   
_________________________________________________________________
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 2

In [17]:
gan.train(itemsAll, epochs=5000, batch_size=32, sample_interval=1)

InternalError:  Blas GEMM launch failed : a.shape=(32, 1024), b.shape=(1024, 256), m=32, n=256, k=1024
	 [[node functional_3/sequential_1/dense_3/MatMul (defined at <ipython-input-15-ff6f850cab9e>:79) ]] [Op:__inference_predict_function_5129]

Function call stack:
predict_function
